In [1]:
import cv2
import random
import os
import matplotlib.pyplot as plt
import numpy as np

dir_path = os.path.dirname(os.path.abspath("photos_augumentation.ipynb"))
dir_path_data = dir_path + "/Data_to_extend"

dir_path_image = dir_path_data + "/images"
dir_path_labels = dir_path_data + "/labels"
dir_path_image_transformed = dir_path_data + "/images_transformed"
dir_path_labels_transformed = dir_path_data + "/labels_transformed"
print(dir_path_labels_transformed)
# os.chdir(dir_path_data)
if(not os.path.exists(dir_path_image)): os.makedirs(dir_path_image)
if(not os.path.exists(dir_path_labels)): os.makedirs(dir_path_labels)
if(not os.path.exists(dir_path_image_transformed)): os.makedirs(dir_path_image_transformed)
if(not os.path.exists(dir_path_labels_transformed)): os.makedirs(dir_path_labels_transformed)
os.chdir(dir_path_image)

files = os.listdir(dir_path_image)
files.sort(key = len)



/home/olson/Desktop/YOLOv9_yeast_and_defects_detection/Data_to_extend/labels_transformed


In [2]:
def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

In [3]:
print(files)

['Capture_4_bmp.rf.70b179a3a4c5eeb16a4cd0878a21855d.jpg', 'Capture_8_bmp.rf.5b4c4d7983b7a1fb97e20e2b11c6a1a1.jpg', 'Capture_7_bmp.rf.4f2497e7b79cd97f4a4b7ff2262a49db.jpg', 'Capture_5_bmp.rf.a25d72d38ae47cb9158baad43a866cd8.jpg', 'Capture_9_bmp.rf.aa63e84e38aeba08637f043d27782d35.jpg', 'Capture_6_bmp.rf.e0216929a93ab729b5b6da7d7cb55c7f.jpg', 'Capture_2_bmp.rf.55874dc486a14db2952f186d723a85b1.jpg', 'Capture_3_bmp.rf.608f2b7ba96af3e230dbef4c9dcb8b9e.jpg', 'Capture_15_bmp.rf.f805e65e2f142ba53f2c8db0d0bbdbcf.jpg', 'Capture_60_bmp.rf.d1c4394a8718bb840937d41e2f2bf4e0.jpg', 'Capture_82_bmp.rf.c2f0a01a8a847fd8f35d4aada1eb744b.jpg', 'Capture_40_bmp.rf.75419655f18bf3146946064928a774b4.jpg', 'Capture_45_bmp.rf.a93c67dda0a8453a4edc5cefcc7a92e9.jpg', 'Capture_13_bmp.rf.d7aced33f462ea352301cbebf72f8a9a.jpg', 'Capture_20_bmp.rf.501e3fecc7fb09bb17c3d4b775b79d91.jpg', 'Capture_36_bmp.rf.ac9ab21deb8bc43aa01ef443fc79ffb2.jpg', 'Capture_53_bmp.rf.2807a2893db3d90a69fde58bd8d83e27.jpg', 'Capture_66_bmp.rf.1d

In [4]:
for i in range (6):    
    for item in files:
        os.chdir(dir_path_image)

        image = cv2.imread(item) # Read image
        dh, dw, _ = image.shape

        # random_number = random.randrange(7) # Select random change
        random_number = i
        if random_number == 0:
            transfered_image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
        elif random_number == 1:
            transfered_image = cv2.rotate(image, cv2.ROTATE_180)
        elif random_number == 2:
            transfered_image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
        elif random_number == 3:
            transfered_image = cv2.flip(image, 1)
        elif random_number == 4:
            transfered_image = cv2.flip(image, 0)
        elif random_number == 5:
            transfered_image = cv2.flip(image, -1)
        else: 
            transfered_image = image

        # alpha = random.uniform(0.975, 1.025)    # Change contast and brightness
        # beta = random.uniform(-10, 10)
        # transfered_image = cv2.convertScaleAbs(transfered_image, alpha=alpha, beta=beta)






        os.chdir(dir_path_labels)

        fl = open(item[:-4] + ".txt", 'r') # Read bound boxes from txt
        data = fl.readlines()
        fl.close()

        bound_boxes = []

        for dt in data: # Do for all bound boxes in txt

        # Split string to float
            label, x, y, w, h = map(float, dt.split(' '))

            l = int((x - w / 2) * dw)
            r = int((x + w / 2) * dw)
            t = int((y - h / 2) * dh)
            b = int((y + h / 2) * dh)
            
            # Check boudaries

            if l < 0:
                l = 0
            if r > dw - 1:
                r = dw - 1
            if t < 0:
                t = 0
            if b > dh - 1:
                b = dh - 1

            bound_boxes.append([int(label), l, r, t, b])

        bound_boxes_to_transform = []

        # Get new boundboxes after photo change
        for item2 in bound_boxes:
            a,x1,x2,y1,y2 = item2

            diff_x = x2 - x1
            diff_y = y2 - y1
            
            if random_number == 0:
                new_x1 = dh - y1 - diff_y
                new_y1 = x1
                new_x2 = dh - y2 + diff_y
                new_y2 = x2
            elif random_number == 1:
                new_x1 = dw - x1 - diff_x
                new_y1 = dh - y1 - diff_y
                new_x2 = dw - x2 + diff_x
                new_y2 = dh - y2 + diff_y
            elif random_number == 2:
                new_x1 = y1
                new_y1 = dw - x1 - diff_x
                new_x2 = y2
                new_y2 = dw - x2 + diff_x
            elif random_number == 3:
                new_x1 = dw - x1 - diff_x
                new_y1 = y1
                new_x2 = dw - x2 + diff_x
                new_y2 = y2
            elif random_number == 4:
                new_x1 = x1
                new_y1 = dh - y1 - diff_y
                new_x2 = x2
                new_y2 = dh - y1
            elif random_number == 5:
                new_x1 = dw - x1 - diff_x
                new_y1 = dh - y1 - diff_y
                new_x2 = dw - x2 + diff_x
                new_y2 = dh - y1
            else: 
                new_x1 = x1
                new_y1 = y1
                new_x2 = x2
                new_y2 = y2
            # Check if photo rotated
            if random_number == 0 or random_number == 2:
                temp = dw
                dw = dh
                dh = temp
        
            bound_boxes_to_transform.append([int(a) , new_x1, new_x2, new_y1, new_y2])

        bound_boxes_to_save = []
        # Create new tab for changed format boxes
        for item3 in bound_boxes_to_transform:
            temp = convert((dw,dh), item3[1::])
            bound_boxes_to_save.append([item3[0], temp[0], temp[1], temp[2], temp[3]])



        # Save new boundboxes cordinates
        os.chdir(dir_path_labels_transformed)
        with open(item[:-4] + "_" + str(i) + "transformed.txt", 'w') as f:
            for item4 in bound_boxes_to_save:
                f.writelines(str(item4[0]) + " " + str(item4[1]) + " " + str(item4[2]) + " " + str(item4[3]) + " " + str(item4[4]) + "\n")




        # Save new images
        os.chdir(dir_path_image_transformed)
        cv2.imwrite(item[:-4] + "_" + str(i) + "transformed.jpg", transfered_image)

In [5]:
#Check bounding boxes

files = os.listdir(dir_path_image_transformed)
files.sort(key = len)

# print(files)

for item in files:
    os.chdir(dir_path_image_transformed)
    image = cv2.imread(item) # Read image
    dh, dw, _ = image.shape

    os.chdir(dir_path_labels_transformed)
    fl = open(item[:-4] + ".txt", 'r')
    data = fl.readlines()
    # print(data)
    fl.close()

    for dt in data:

        # Split string to float
        label, x, y, w, h = map(float, dt.split(' '))

        l = int((x - w / 2) * dw)
        r = int((x + w / 2) * dw)
        t = int((y - h / 2) * dh)
        b = int((y + h / 2) * dh)
        
        # Check boudaries

        if l < 0:
            l = 0
        if r > dw - 1:
            r = dw - 1
        if t < 0:
            t = 0
        if b > dh - 1:
            b = dh - 1

        bound_boxes.append([int(label), l, r, t, b])
        cv2.rectangle(image, (l, t), (r, b), (0, 0, 255), 1)

    # plt.imshow(image)
    # plt.show()